In [ ]:
import os
import gzip
import requests
import numpy as np
import netCDF4 as nc
from collections import OrderedDict as OD

#### Fetching the Location of a place in `Berlin, Germany`

- Longitude of the location is - 13.3946
- Latitute of the location is - 52.5246

In [ ]:
def create_dir(dir_name):
    try:
        os.mkdir(dir_name)
        print(dir_name, "Dir Created")
    except FileExistsError:
        print(dir_name, "Dir Already exists")
    return

def fetch_decompress(url_param, file_name):
    url = f'https://opendata.dwd.de/climate_environment/CDC/grids_germany/monthly/Project_TRY/{url_param}/{file_name}.gz'
    with open(f'./{url_param}/{file_name}', 'wb+') as f:
        print('Fetching URL: ', url)
        payload = requests.get(url)
        print("Fetched", file_name)
        dec_payload = gzip.decompress(payload.content)
        print("Decompressed", file_name)
        f.write(dec_payload)
        print('Completed Writing', file_name)
    return

def delete_file(url_param, file_name):
    if os.path.exists(f"./{url_param}/{file_name}"):
        os.remove(f"./{url_param}/{file_name}")
        print(f"Deleted '/{url_param}/{file_name}'")
    else:
        print(f"The file '/{url_param}/{file_name}' does not exist")
    return

In [ ]:
def getclosest_ij(lats,lons,latpt,lonpt):
    # find squared distance of every point on grid
    dist_sq = (lats-latpt)**2 + (lons-lonpt)**2  
    # 1D index of minimum dist_sq element
    minindex_flattened = dist_sq.argmin()    
    # Get 2D index for latvals and lonvals arrays from 1D index
    return np.unravel_index(minindex_flattened, lats.shape)

In [ ]:
def get_value(url_param, file_name, latpt, lonpt):
    output_params = {'air_temperature_mean' : 'temperature',
                     'pressure' : 'SLP',
                     'wind_direction' : 'DD',
                     'wind_speed' : 'FF'}
    
    output_variable = output_params[url_param]
    
    with nc.Dataset(f'./{url_param}/{file_name}') as ncfile:
        print(f"Opened netCDF: {file_name}")
        lon = f.variables['lon']
        lat = f.variables['lat']
        temp = f.variables[output_variable]
        y, x = getclosest_ij(lat[:], lon[:], latpt, lonpt)
        value = float(temp[0, y, x].data)
        print(f"Got the Value {value} from {file_name}")
        return 

In [ ]:
params = OD({'air_temperature_mean' : 'TT',
'pressure' : 'PRED',
'wind_direction' : 'DD',
'wind_speed' : 'FF'})

In [ ]:
lon, lat = 13.3946, 52.5246

In [ ]:
start_year = 1995
end_year = 2004
start_month = 1
end_month = 12

csvfile = open('./dataset.csv', 'w+')
csvfile.write(','.join([i for i in params]))

for year in range(start_year, end_year+1):
    for month in range(start_month, end_month+1):
        value_list = []
        for url_param, measurement_param in params.items():
            print(f"Fetching {url_param} of Year: {year} of Month: {month}")
            file_name = f'{measurement_param}_{year}{month:02d}_monmean.nc'
            create_dir(url_param)
            fetch_decompress(url_param, file_name)
            value = get_value(url_param, file_name, lat, lon)
            value_list.append(str(value))
        value_list.append('\n')
        csv_row = ','.join(value_list)
        print(csv_row)
        csvfile.write(csv_row)
        delete_file(url_param, file_name)
    break
    
csvfile.close()